## SOURCE TO SOURCE LAYER

> This Notebook reads the RAW files and performs data harmonization.


In [1]:
# Importing Common Utility Function
import pandas as pd

from StockETL import GlobalPath

In [2]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [3]:
# Instantiate GlobalPath
symbol_source_layer_path = GlobalPath("DATA/SOURCE/Symbol")
symbol_bronze_layer_path = GlobalPath("DATA/BRONZE/Symbol/Symbol_data.csv")
symbol_bronze_schema_file_path = GlobalPath("CONFIG/DATA_CONTRACTS/BRONZE/Symbol.json")

### Define a function to read and process an CSV file


In [4]:
# Define a function to read and process an csv file


def read_file(file_path: GlobalPath) -> None:
    """
    Processes CSV files from the SOURCE layer and consolidates them into a single DataFrame.
    The data is then harmonized and saved as a CSV file in the BRONZE layer.
    """
    # Log the reading of the file
    print(f"Processing file => {file_path}")

    # Read each CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Harmonize the DataFrame by replacing punctuation from column names
    df = replace_punctuation_from_columns(df)

    # Drop rows where 'isin' is NaN or null
    df = df.dropna(subset=["isin"])

    # Align Datafame with DataContract
    df = align_with_datacontract(df, symbol_bronze_schema_file_path)

    # Drop columns where all elements are NaN
    df.dropna(how="all", axis=1, inplace=True)
    return df

In [5]:
# Initialize an empty list to store DataFrames
df_symbol_list = []
# Generate file paths for available Excel files in the source layer
file_paths = check_files_availability(symbol_source_layer_path, file_pattern="*.csv")

# Loop through all CSV files in the SOURCE layer folder
for file_path in file_paths:
    try:
        df = read_file(file_path)
        # Append the DataFrame to the list
        df_symbol_list.append(df)
    except Exception as e:
        # Log any exceptions during file reading
        print(f"Failed to read {file_path} due to error => {e}")

# Concatenate all DataFrames into one
df = pd.concat(df_symbol_list, ignore_index=True)

# Align Datafame with DataContract
df = align_with_datacontract(df, symbol_bronze_schema_file_path)

# Save the result as a CSV file in the BRONZE layer
df.to_csv(symbol_bronze_layer_path, index=None)
print(
    f"Successfully created BRONZE Layer CSV file for Symbol at: {symbol_bronze_layer_path}"
)
# Log the DataFrame debugrmation
df.info()

Number of Files Detected => 4
Processing file => C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SOURCE\Symbol\AMFI_Reports.csv
DataContract loaded from => C:\Users\prashant.tripathi\Code\PortfolioTracker\CONFIG\DATA_CONTRACTS\BRONZE\Symbol.json
Processing file => C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SOURCE\Symbol\EQUITY_Symbols.csv
DataContract loaded from => C:\Users\prashant.tripathi\Code\PortfolioTracker\CONFIG\DATA_CONTRACTS\BRONZE\Symbol.json
Processing file => C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SOURCE\Symbol\INDEX_Symbols.csv
DataContract loaded from => C:\Users\prashant.tripathi\Code\PortfolioTracker\CONFIG\DATA_CONTRACTS\BRONZE\Symbol.json
Processing file => C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\SOURCE\Symbol\Mannual_Symbols.csv
DataContract loaded from => C:\Users\prashant.tripathi\Code\PortfolioTracker\CONFIG\DATA_CONTRACTS\BRONZE\Symbol.json
DataContract loaded from => C:\Users\prashant.tripathi\Code\PortfolioTracker

Successfully created BRONZE Layer CSV file for Symbol at: C:\Users\prashant.tripathi\Code\PortfolioTracker\DATA\BRONZE\Symbol\Symbol_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16926 entries, 0 to 16925
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   instrument_type    16926 non-null  string
 1   symbol             4343 non-null   string
 2   scrip_name         16926 non-null  string
 3   scrip_code         4343 non-null   string
 4   isin               16926 non-null  string
 5   isin_reinvestment  3449 non-null   string
dtypes: string(6)
memory usage: 793.5 KB
